### Задачи 1 и 2
Контрольная работа состоит из пяти вопросов. На каждый вопрос приведено четыре варианта ответа, один из которых правильный. Случайная величина X задаёт число правильных ответов при простом угадывании. Найдите математическое ожидание данной случайной величины. Найдите дисперсию случайной величины X.

Подсказка: постройте закон распределения случайной величины X.

In [2]:
import numpy as np

In [3]:
def combinations(n: int, k: int) -> int:
    """Число сочетаний.
    """
    
    return np.math.factorial(n) // (np.math.factorial(k) * np.math.factorial(n - k))

In [38]:
def bernuli(n,k,p):
    return combinations(n,k) * (p**k) * ((1-p)**(n-k))

Таким образом наши случайные величины будут соответсвовать количеству верных ответов: $x_1 = 0$, $x_2 = 1$, $x_3 = 2$, $x_4 = 3$, $x_5 = 4$, $x_6 = 5$


По формуле Бернули найдем вероятности для каждого исхода.

In [39]:
for i in range(6):
    print(f"Вероятность {i} из 5 верных ответов, {bernuli(5,i,0.25)}")

Вероятность 0 из 5 верных ответов, 0.2373046875
Вероятность 1 из 5 верных ответов, 0.3955078125
Вероятность 2 из 5 верных ответов, 0.263671875
Вероятность 3 из 5 верных ответов, 0.087890625
Вероятность 4 из 5 верных ответов, 0.0146484375
Вероятность 5 из 5 верных ответов, 0.0009765625


Находим мат. ожидание по формуле мат. ожидания биноминального распределения: M(X) = np

In [40]:
MX = 5*0.25
MX

1.25

Т.е. при случайном выборе ответов, мы ожидаем получить 1.25 верных.

Далее ищем дисперсию по формуле дисперсии биноминального распределения D(X) = np(1-p)

In [41]:
DX = 1.25*(1-0.25)
DX

0.9375

#### Ответ: Мат ожидание = 1.25, Дисперсия = 0.9375

*Тоже самое, если посчитать, как просто распределение случайной величины.

In [42]:
vars = np.array([bernuli(5,i,0.25) for i in range(6)]) #Вероятности исходов
vars

array([0.23730469, 0.39550781, 0.26367188, 0.08789062, 0.01464844,
       0.00097656])

In [43]:
x = np.array([i for i in range(6)])  # Исходы
x

array([0, 1, 2, 3, 4, 5])

In [44]:
M = (vars*x).sum() #Мат ожидание
M

1.25

In [45]:
xstd = (x - M)**2 #Квадрат откл от мат ожидания
xstd

array([ 1.5625,  0.0625,  0.5625,  3.0625,  7.5625, 14.0625])

In [46]:
D = (xstd*vars).sum() #Дисперсия
D

0.9375

### Задача 3
Пользователь получает в среднем 10 писем со спамом на свой почтовый ящик за сутки. Найти число N, такое, что с вероятностью 0.95 пользователь получит не более N писем со спамом за текущий день.

In [47]:
def poisson_proba(k: int, lambda_: float) -> float:
    """Формула Пуассона.
    """
    
    return (lambda_ ** k) * (np.exp(-lambda_)) / np.math.factorial(k)

У нас получается, что $P(X \leq N) = \displaystyle\sum_{i = 0}^{N} \dfrac{10^{i} e^{-10}}{i!} = 0.95$

Вероятно как-то можно выразить N из формулы, но я не осилил, потому, просто написал код для итеративного решения.

In [48]:
lambda_ = 10

In [49]:
j=0
while sum(poisson_proba(k=i, lambda_=lambda_) for i in range(j)) < 0.95:
    j+=1
print(j)

16


In [50]:
sum(poisson_proba(k=i, lambda_=lambda_) for i in range(16))

0.9512595966960214

#### Ответ: N = 15

### Задача 4
Производятся выстрелы по мишени. Вероятность попадания в мишень при одном выстреле равна 0.01. Сколько выстрелов нужно сделать чтобы быть уверенным с вероятностью 0.9, что хотя бы 6 раз будет совершено попадание?

Подсказка: 1) "Вероятность попасть k раз при n выстрелах" - на какое распределение это похоже? 2) А если нам нужна вероятность P(X >= k), а не P(X = k)? 3) Здесь предстоит немножко покодить.

Будем отталкиваться от биноминального распределения. У нас получается, что "хотябы 6 раз" - это вероятность больше 5 попаданий. P(X>5) = 1 - P(X<=5)

In [114]:
n=6
while True:
    P5 = 0
    for k in range(6):
        P5 += bernuli(n,k,0.01)
    P = 1 - P5
    print(f"{n} - {P}")
    if  P > 0.9:
        print(f"Ответ: Для уверенности 0.9, что мы попадем хотябы 6 раз, нам нужно сделать {n} выстрелов")
        break
        
    n+=1


6 - 9.999778782798785e-13
7 - 6.940115149234316e-12
8 - 2.752198469124778e-11
9 - 8.185885302935958e-11
10 - 2.028940349063646e-10
11 - 4.42543446332877e-10
12 - 8.775072890543356e-10
13 - 1.6157031268448918e-09
14 - 2.8032755006179855e-09
15 - 4.632137118321111e-09
16 - 7.347996544559976e-09
17 - 1.1258834087257696e-08
18 - 1.67437835818518e-08
19 - 2.4262383657713826e-08
20 - 3.436415985103025e-08
21 - 4.769850470154324e-08
22 - 6.502481897374679e-08
23 - 8.722288513673959e-08
24 - 1.1530343868582094e-07
25 - 1.5041890988332085e-07
26 - 1.9387430538753136e-07
27 - 2.471382045676762e-07
28 - 3.1185384186116494e-07
29 - 3.8985025407889395e-07
30 - 4.831534615723498e-07
31 - 5.939976721602847e-07
32 - 7.248364730649826e-07
33 - 8.78353999533843e-07
34 - 1.0574760563875074e-06
35 - 1.2653811746199395e-06
36 - 1.5055115861750679e-06
37 - 1.7815840982482456e-06
38 - 2.097600851791448e-06
39 - 2.45785995078851e-06
40 - 2.866965944980926e-06
41 - 3.3298401560521995e-06
42 - 3.851730828952071

#### Ответ: Для уверенности 0.9, что мы попадем хотябы 6 раз, нам нужно сделать 926 выстрелов